In [1]:
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score, accuracy_score, mean_absolute_error, f1_score
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from tqdm import tqdm
import lightgbm as lgb
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
from datetime import datetime
from sklearn.preprocessing import PolynomialFeatures
from itertools import combinations

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
seed = 2020

In [3]:
df_train = pd.read_csv('./data/used_car_train_20200313.csv', sep=' ')
df_test = pd.read_csv('./data/used_car_testB_20200421.csv', sep=' ')
df_sub = pd.read_csv('./data/used_car_sample_submit.csv', sep=' ')

In [4]:
df_feature = pd.concat([df_train, df_test], sort=False)

In [5]:
df_feature['regionCode'].max()

8120

In [6]:
df_feature.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,offerType,creatDate,price,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,0.0,1046,0,0,20160404,1850.0,43.357796,3.966344,0.050257,2.159744,1.143786,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,-,4366,0,0,20160309,3600.0,45.305273,5.236112,0.137925,1.380657,-1.422165,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,0.0,2806,0,0,20160402,6222.0,45.978359,4.823792,1.319524,-0.998467,-0.996911,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,0.0,434,0,0,20160312,2400.0,45.687478,4.492574,-0.050616,0.883600,-2.228079,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,0.0,6977,0,0,20160313,5200.0,44.383511,2.031433,0.572169,-1.571239,2.246088,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482


In [7]:
from scipy import stats

cols = ['bodyType', 'fuelType', 'gearbox']
df_feature['gp'] = df_feature['brand'].astype(
    'str') + df_feature['model'].astype('str')
gp_col = 'gp'

df_na = df_feature[cols].isna()
# 根据分组计算众数
df_mode = df_feature.groupby(gp_col)[cols].agg(
    lambda x: stats.mode(x)[0][0])

for col in cols:
    na_series = df_na[col]
    names = list(df_feature.loc[na_series, gp_col])

    t = df_mode.loc[names, col]
    t.index = df_feature.loc[na_series, col].index

    df_feature.loc[na_series, col] = t

del df_feature['gp']
df_feature[cols].isnull().sum()

bodyType    15
fuelType    41
gearbox      3
dtype: int64

In [8]:
df_feature['notRepairedDamage'] = df_feature['notRepairedDamage'].replace(
    '-', 2)
df_feature['notRepairedDamage'] = df_feature['notRepairedDamage'].astype(
    'float')

In [9]:
df_feature.tail()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,offerType,creatDate,price,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
49995,249995,111443,20041005,4.0,4,0.0,0.0,1.0,150,15.0,2.0,5564,0,0,20160309,NaN,46.321013,-3.304401,0.073363,-0.622359,-0.778349,0.263668,0.000292,0.141804,0.076393,0.039272,2.072901,-2.531869,1.716978,-1.063437,0.326587
49996,249996,152834,20130409,65.0,1,0.0,0.0,0.0,179,4.0,0.0,5220,0,0,20160323,NaN,48.086547,-3.318641,0.965881,-2.672160,0.357440,0.255310,0.000991,0.155868,0.108425,0.067841,1.358504,-3.290295,4.269809,0.140524,0.556221
49997,249997,132531,20041211,4.0,4,0.0,0.0,1.0,147,12.5,0.0,3795,0,0,20160316,NaN,46.145279,-3.305263,-0.015283,-0.288329,-0.687112,0.262933,0.000318,0.141872,0.071968,0.042966,2.165658,-2.417885,1.370612,-1.073133,0.270602
49998,249998,143405,20020702,40.0,1,4.0,0.0,1.0,176,15.0,0.0,61,0,0,20160327,NaN,45.507088,-3.197006,-1.141252,-0.434930,-1.845040,0.282106,0.000023,0.067483,0.067526,0.009006,2.030114,-2.939244,0.569078,-1.718245,0.316379
49999,249999,78202,20090708,32.0,8,1.0,0.0,0.0,0,3.0,0.0,4158,0,0,20160401,NaN,44.289471,4.181452,0.547068,-0.775841,1.789601,0.231449,0.103947,0.096027,0.062328,0.110180,-3.689090,2.032376,0.109157,2.202828,0.847469


In [10]:
del df_feature['seller']
del df_feature['offerType']

In [11]:
df_feature['price'] = np.log1p(df_feature['price'])

# feature engine

In [12]:
v_cols = ['v_'+str(i) for i in range(15)]

df_feature['v_mean'] = df_feature[v_cols].mean(axis=1)
df_feature['v_max'] = df_feature[v_cols].max(axis=1)
df_feature['v_min'] = df_feature[v_cols].min(axis=1)
df_feature['v_std'] = df_feature[v_cols].std(axis=1)

for col in ['v_mean', 'v_max', 'v_min', 'v_std']:
    df_feature[f'name_{col}_mean'] = df_feature.groupby('name')[
        col].transform('mean')
    df_feature[f'name_{col}_std'] = df_feature.groupby('name')[
        col].transform('std')
    df_feature[f'name_{col}_max'] = df_feature.groupby('name')[
        col].transform('max')
    df_feature[f'name_{col}_min'] = df_feature.groupby('name')[
        col].transform('min')

In [13]:
# # 时序特征
# df_sort = df_feature[['brand', 'model', 'creatDate', 'price']]
# df_sort = df_sort.groupby(['brand', 'model', 'creatDate'])[
#     'price'].mean().reset_index()
# df_sort.rename(columns={'price': 'brand_model_day_price_mean'}, inplace=True)
# df_sort = df_sort.sort_values(['brand', 'model', 'creatDate'])
# df_sort['brand_model_day_shift1_price_mean'] = df_sort.groupby(
#     ['brand', 'model'])['brand_model_day_price_mean'].shift(1)
# del df_sort['brand_model_day_price_mean']
# # # df_sort['brand_model_price_rolling3_mean'] = df_sort.groupby(
# # #     ['brand', 'model'])['price'].shift().rolling(window=3, min_periods=3).mean()
# # df_sort['brand_model_price_shift1'] = df_sort.groupby(
# #     ['brand', 'model'])['price'].shift()
# # df_sort.head()
# # print(df_feature.shape)
# df_feature = df_feature.merge(df_sort, how='left')
# # print(df_feature.shape)

In [14]:
df_feature['name_count'] = df_feature.groupby(
    ['name'])['SaleID'].transform('count')

In [15]:
def date_parse(x):
    year = int(str(x)[:4])
    month = int(str(x)[4:6])
    day = int(str(x)[6:8])

    if month < 1:
        month = 1

    date = datetime(year, month, day)
    return date


df_feature['regDate'] = df_feature['regDate'].apply(date_parse)
df_feature['creatDate'] = df_feature['creatDate'].apply(date_parse)
df_feature['regDate_year'] = df_feature['regDate'].dt.year
df_feature['creatDate_year'] = df_feature['creatDate'].dt.year
df_feature['creatDate_month'] = df_feature['creatDate'].dt.month

In [16]:
df_feature['car_age_day'] = (
    df_feature['creatDate'] - df_feature['regDate']).dt.days
df_feature['car_age_year'] = round(df_feature['car_age_day'] / 365, 1)

In [17]:
# 简单统计
def stat(df, df_merge, group_by, agg):
    group = df.groupby(group_by).agg(agg)

    columns = []
    for on, methods in agg.items():
        for method in methods:
            columns.append('{}_{}_{}'.format('_'.join(group_by), on, method))
    group.columns = columns
    group.reset_index(inplace=True)
    df_merge = df_merge.merge(group, on=group_by, how='left')

    del (group)
    gc.collect()

    return df_merge

In [18]:
l = ['name', 'model', 'brand', 'bodyType']
for f1 in tqdm(l):
    for f2 in v_cols:
        df_feature = stat(df_feature, df_feature, [f1], {
            f2: ['mean', 'max', 'min', 'std']})

100%|██████████| 4/4 [00:51<00:00, 12.76s/it]


In [19]:
def statis_feat(df_know, df_unknow):
    l = ['name', 'model', 'brand', 'bodyType']

    combs = list(combinations(l, 2))
    for t1, t2 in combs:
        df_unknow = stat(df_know, df_unknow, [t1, t2], {
                         'price': ['mean', 'max', 'min', 'std', 'median']})

    for f in tqdm(l):
        df_unknow = stat(df_know, df_unknow, [f], {
                         'price': ['mean', 'max', 'min', 'std']})

    return df_unknow

In [20]:
# 5折交叉
df_train = df_feature[~df_feature['price'].isnull()]
df_train = df_train.reset_index(drop=True)
df_test = df_feature[df_feature['price'].isnull()]

df_stas_feat = None
kf = KFold(n_splits=5, random_state=seed, shuffle=True)
for train_index, val_index in kf.split(df_train):
    df_fold_train = df_train.iloc[train_index]
    df_fold_val = df_train.iloc[val_index]

    df_fold_val = statis_feat(df_fold_train, df_fold_val)
    df_stas_feat = pd.concat([df_stas_feat, df_fold_val], axis=0)

    del(df_fold_train)
    del(df_fold_val)
    gc.collect()

df_test = statis_feat(df_train, df_test)
df_feature = pd.concat([df_stas_feat, df_test], axis=0)

del(df_stas_feat)
del(df_train)
del(df_test)
gc.collect()

100%|██████████| 4/4 [00:01<00:00,  2.63it/s]


27

In [21]:
df_feature['v_0_add_v_4'] = df_feature['v_0'] + df_feature['v_4']
df_feature['v_0_add_v_8'] = df_feature['v_0'] + df_feature['v_8']
df_feature['v_1_add_v_3'] = df_feature['v_1'] + df_feature['v_3']
df_feature['v_1_add_v_4'] = df_feature['v_1'] + df_feature['v_4']
df_feature['v_1_add_v_5'] = df_feature['v_1'] + df_feature['v_5']
df_feature['v_1_add_v_12'] = df_feature['v_1'] + df_feature['v_12']
df_feature['v_2_add_v_3'] = df_feature['v_2'] + df_feature['v_3']
df_feature['v_4_add_v_11'] = df_feature['v_4'] + df_feature['v_11']
df_feature['v_4_add_v_12'] = df_feature['v_4'] + df_feature['v_12']
df_feature['v_0_add_v_12_add_v_14'] = df_feature['v_0'] + \
    df_feature['v_12'] + df_feature['v_14']

In [22]:
df_feature['v_4_add_v_9_minu_v_13'] = df_feature['v_4'] + \
    df_feature['v_9'] - df_feature['v_13']
df_feature['v_2_add_v_4_minu_v_11'] = df_feature['v_2'] + \
    df_feature['v_4'] - df_feature['v_11']
df_feature['v_2_add_v_3_minu_v_11'] = df_feature['v_2'] + \
    df_feature['v_3'] - df_feature['v_11']

In [23]:
df_feature.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,creatDate,price,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,v_mean,v_max,v_min,v_std,name_v_mean_mean,name_v_mean_std,name_v_mean_max,name_v_mean_min,name_v_max_mean,name_v_max_std,name_v_max_max,name_v_max_min,name_v_min_mean,name_v_min_std,name_v_min_max,name_v_min_min,name_v_std_mean,name_v_std_std,name_v_std_max,name_v_std_min,name_count,regDate_year,creatDate_year,creatDate_month,car_age_day,car_age_year,name_v_0_mean,name_v_0_max,name_v_0_min,name_v_0_std,name_v_1_mean,name_v_1_max,name_v_1_min,name_v_1_std,name_v_2_mean,name_v_2_max,name_v_2_min,name_v_2_std,name_v_3_mean,name_v_3_max,name_v_3_min,name_v_3_std,name_v_4_mean,name_v_4_max,name_v_4_min,name_v_4_std,name_v_5_mean,name_v_5_max,name_v_5_min,name_v_5_std,name_v_6_mean,name_v_6_max,name_v_6_min,name_v_6_std,name_v_7_mean,name_v_7_max,name_v_7_min,name_v_7_std,name_v_8_mean,name_v_8_max,name_v_8_min,name_v_8_std,name_v_9_mean,name_v_9_max,name_v_9_min,name_v_9_std,name_v_10_mean,name_v_10_max,name_v_10_min,name_v_10_std,name_v_11_mean,name_v_11_max,name_v_11_min,name_v_11_std,name_v_12_mean,name_v_12_max,name_v_12_min,name_v_12_std,name_v_13_mean,name_v_13_max,name_v_13_min,name_v_13_std,name_v_14_mean,name_v_14_max,name_v_14_min,name_v_14_std,model_v_0_mean,model_v_0_max,model_v_0_min,model_v_0_std,model_v_1_mean,model_v_1_max,model_v_1_min,model_v_1_std,model_v_2_mean,model_v_2_max,model_v_2_min,model_v_2_std,model_v_3_mean,model_v_3_max,model_v_3_min,model_v_3_std,model_v_4_mean,model_v_4_max,model_v_4_min,model_v_4_std,model_v_5_mean,model_v_5_max,model_v_5_min,model_v_5_std,model_v_6_mean,model_v_6_max,model_v_6_min,model_v_6_std,model_v_7_mean,model_v_7_max,model_v_7_min,model_v_7_std,model_v_8_mean,model_v_8_max,model_v_8_min,model_v_8_std,model_v_9_mean,model_v_9_max,model_v_9_min,model_v_9_std,model_v_10_mean,model_v_10_max,model_v_10_min,model_v_10_std,model_v_11_mean,model_v_11_max,model_v_11_min,model_v_11_std,model_v_12_mean,model_v_12_max,model_v_12_min,model_v_12_std,model_v_13_mean,model_v_13_max,model_v_13_min,model_v_13_std,model_v_14_mean,model_v_14_max,model_v_14_min,model_v_14_std,brand_v_0_mean,brand_v_0_max,brand_v_0_min,brand_v_0_std,brand_v_1_mean,brand_v_1_max,brand_v_1_min,brand_v_1_std,brand_v_2_mean,brand_v_2_max,brand_v_2_min,brand_v_2_std,brand_v_3_mean,brand_v_3_max,brand_v_3_min,brand_v_3_std,brand_v_4_mean,brand_v_4_max,brand_v_4_min,brand_v_4_std,brand_v_5_mean,brand_v_5_max,brand_v_5_min,brand_v_5_std,brand_v_6_mean,brand_v_6_max,brand_v_6_min,brand_v_6_std,brand_v_7_mean,brand_v_7_max,brand_v_7_min,brand_v_7_std,brand_v_8_mean,brand_v_8_max,brand_v_8_min,brand_v_8_std,brand_v_9_mean,brand_v_9_max,brand_v_9_min,brand_v_9_std,brand_v_10_mean,brand_v_10_max,brand_v_10_min,brand_v_10_std,brand_v_11_mean,brand_v_11_max,brand_v_11_min,brand_v_11_std,brand_v_12_mean,brand_v_12_max,brand_v_12_min,brand_v_12_std,brand_v_13_mean,brand_v_13_max,brand_v_13_min,brand_v_13_std,brand_v_14_mean,brand_v_14_max,brand_v_14_min,brand_v_14_std,bodyType_v_0_mean,bodyType_v_0_max,bodyType_v_0_min,bodyType_v_0_std,bodyType_v_1_mean,bodyType_v_1_max,bodyType_v_1_min,bodyType_v_1_std,bodyType_v_2_mean,bodyType_v_2_max,bodyType_v_2_min,bodyType_v_2_std,bodyType_v_3_mean,bodyType_v_3_max,bodyType_v_3_min,bodyType_v_3_std,bodyType_v_4_mean,bodyType_v_4_max,bodyType_v_4_min,bodyType_v_4_std,bodyType_v_5_mean,bodyType_v_5_max,bodyType_v_5_min,bodyType_v_5_std,bodyType_v_6_mean,bodyType_v_6_max,bodyType_v_6_min,bodyType_v_6_std,bodyType_v_7_mean,bodyType_v_7_max,bodyType_v_7_min,bodyType_v_7_std,bodyType_v_8_mean,bodyType_v_8_max,bodyType_v_8_min,bodyType_v_8_std,bodyType_v_9_mean,bodyType_v_9_max,bodyType_v_9_min,bodyType_v_9_std,bodyType_v_10_mean,bodyType_v_10_max,bodyType_v_10_min,bodyType_v_10_std,bodyType_v_11_mean,bodyType_v_11_max,bodyType_v_11_min,bodyType_v_11_std,bodyType_v_12_mean,bodyType_v_12_max,bodyType_v_12_min,bodyTy

In [24]:
df_feature.shape

(200000, 354)

In [25]:
df_feature.to_pickle('feature.pickle')